In [7]:
import pandas as pd
import spacy

file_path = '/usr/colab/bin/Excel/Attribution_scores.xlsx'
df = pd.read_excel(file_path)

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Lexical analysis on a token
def lexical_analysis(token):
    if not isinstance(token, str):
        token = str(token)
    doc = nlp(token)
    analysis = {
        "Token": [],
        "POS": [],
        "Lemma": [],
        "Entity": [],
        "Entity Type": []
    }
    for t in doc:
        analysis["Token"].append(t.text)
        analysis["POS"].append(t.pos_)
        analysis["Lemma"].append(t.lemma_)
        if t.ent_type_:
            analysis["Entity"].append(t.text)
            analysis["Entity Type"].append(t.ent_type_)
    return analysis

# Apply the lexical analysis to each token in the DataFrame
lexical_data = []

for index, row in df.iterrows():
    token = row['Prompt']
    analysis = lexical_analysis(token)
    for i in range(len(analysis["Token"])):
        lexical_data.append({
            "Model": row["Model"],
            "Response": row["Response"],
            "Prompt": row["Prompt"],
            "Attribution Score": row["Score"],
            "Token": analysis["Token"][i],
            "POS": analysis["POS"][i],
            "Lemma": analysis["Lemma"][i],
            "Entity": analysis["Entity"][i] if i < len(analysis["Entity"]) else None,
            "Entity Type": analysis["Entity Type"][i] if i < len(analysis["Entity Type"]) else None
        })

lexical_df = pd.DataFrame(lexical_data)

output_file_path = '/usr/colab/bin/Excel/Lexical_analysis_prompts.xlsx'
lexical_df.to_excel(output_file_path, index=False)

print(f'Lexical analysis saved to {output_file_path}')

Lexical analysis saved to /usr/colab/bin/Excel/Lexical_analysis_prompts.xlsx


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

lexical_file_path = '/usr/colab/bin/Excel/Lexical_analysis_prompts.xlsx'
lexical_df = pd.read_excel(lexical_file_path)

print(lexical_df.describe())

# Count the occurrences of each POS tag
pos_counts = lexical_df['POS'].value_counts()
print(pos_counts)

# Count the occurrences of each entity type
entity_type_counts = lexical_df['Entity Type'].value_counts()
print(entity_type_counts)

# Average attribution score by POS tag
avg_score_by_pos = lexical_df.groupby('POS')['Attribution Score'].mean()
print(avg_score_by_pos)

# Average attribution score by entity type
avg_score_by_entity = lexical_df.groupby('Entity Type')['Attribution Score'].mean()
print(avg_score_by_entity)

# Count of tokens per model
tokens_per_model = lexical_df.groupby('Model').size()
print(tokens_per_model)

# Average attribution score per model
avg_score_per_model = lexical_df.groupby('Model')['Attribution Score'].mean()
print(avg_score_per_model)

# POS tag distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=lexical_df, x='POS', order=lexical_df['POS'].value_counts().index)
plt.title('POS Tag Distribution')
plt.xticks(rotation=45)
plt.savefig('/usr/colab/bin/Excel/pos_tag_distribution_promts.png')
plt.close()

# Average attribution score by POS tag
plt.figure(figsize=(10, 6))
avg_score_by_pos.plot(kind='bar')
plt.title('Average Attribution Score by POS Tag')
plt.xlabel('POS Tag')
plt.ylabel('Average Attribution Score')
plt.xticks(rotation=45)
plt.savefig('/usr/colab/bin/Excel/avg_score_by_pos_promts.png')
plt.close()

# Average attribution score by entity type
plt.figure(figsize=(10, 6))
avg_score_by_entity.plot(kind='bar')
plt.title('Average Attribution Score by Entity Type')
plt.xlabel('Entity Type')
plt.ylabel('Average Attribution Score')
plt.xticks(rotation=45)
plt.savefig('/usr/colab/bin/Excel/avg_score_by_entity_promts.png')
plt.close()

# Average attribution score per model
plt.figure(figsize=(10, 6))
avg_score_per_model.plot(kind='bar')
plt.title('Average Attribution Score per Model')
plt.xlabel('Model')
plt.ylabel('Average Attribution Score')
plt.xticks(rotation=45)
plt.savefig('/usr/colab/bin/Excel/avg_score_per_model_promts.png')
plt.close()

       Attribution Score
count        2529.000000
mean            3.084553
std             5.545881
min           -11.198800
25%             1.054100
50%             1.844200
75%             3.781000
max            39.013900
POS
PUNCT    546
VERB     427
NOUN     393
PRON     309
PROPN    211
AUX      103
ADJ       90
ADV       89
ADP       79
SPACE     69
PART      52
NUM       44
CCONJ     42
X         24
INTJ      23
SCONJ     19
SYM        5
DET        4
Name: count, dtype: int64
Entity Type
ORG         48
PERSON      42
GPE         32
CARDINAL    21
ORDINAL     20
DATE        16
LANGUAGE     8
NORP         6
LAW          2
Name: count, dtype: int64
POS
ADJ      3.954572
ADP      1.459161
ADV      1.893254
AUX      2.688559
CCONJ    3.064188
DET      1.637975
INTJ     3.034622
NOUN     3.760212
NUM      3.630470
PART     2.846104
PRON     1.772848
PROPN    3.796934
PUNCT    3.276727
SCONJ    2.991900
SPACE    3.873259
SYM      0.718080
VERB     3.270722
X        1.305717
Name: Attr

In [9]:
import pandas as pd

file_path = '/usr/colab/bin/Excel/Lexical_analysis_prompts.xlsx'
df = pd.read_excel(file_path)

# Top and bottom 25 attribution scores for each model
def get_top_bottom_attributions(df):
    models = df['Model'].unique()
    result = pd.DataFrame()

    for model in models:
        model_df = df[df['Model'] == model]
        top_25 = model_df.nlargest(25, 'Attribution Score')
        bottom_25 = model_df.nsmallest(25, 'Attribution Score')
        result = pd.concat([result, top_25, bottom_25], ignore_index=True)

    return result

top_bottom_attributions = get_top_bottom_attributions(df)

output_file_path = '/usr/colab/bin/Excel/top_bottom_attributions_promts.xlsx'
top_bottom_attributions.to_excel(output_file_path, index=False)

print("The top and bottom 25 attribution scores for each model have been saved to:", output_file_path)

The top and bottom 25 attribution scores for each model have been saved to: /usr/colab/bin/Excel/top_bottom_attributions_promts.xlsx


In [10]:
# Group by Lemmas and Models, calculate occurrences and summary statistics
grouped_data = lexical_df.groupby(['Model', 'Lemma']).agg({
    'Attribution Score': ['count', 'min', 'max', 'mean']
}).reset_index()

# Rename columns
grouped_data.columns = ['Model', 'Lemma', 'Occurrences', 'Min Score', 'Max Score', 'Average Score']

# Find the top 50 occurring lemmas per model
top_lemmas_per_model = grouped_data.groupby('Model').apply(lambda x: x.nlargest(50, 'Occurrences')).reset_index(drop=True)

output_file_path = '/usr/colab/bin/Excel/Top_lemmas_analysis.xlsx'
top_lemmas_per_model.to_excel(output_file_path, index=False)

print(f'Top lemmas analysis saved to {output_file_path}')

Top lemmas analysis saved to /usr/colab/bin/Excel/Top_lemmas_analysis.xlsx


In [11]:
file_path = '/usr/colab/bin/Excel/Top_lemmas_analysis.xlsx'
df_top_lemmas = pd.read_excel(file_path)

print("Top 50 Lemmas per Model:")
print(df_top_lemmas)

# Average attribution scores across all models grouped by Lemma
lemma_avg_scores = df_top_lemmas.groupby('Lemma')['Average Score'].mean().reset_index()

# Sort by average score
top_high_impact_lemmas = lemma_avg_scores.sort_values(by='Average Score', ascending=False).head(25)

print("\nTop 25 Lemmas with Highest Average Attribution Scores:")
print(top_high_impact_lemmas[['Lemma', 'Average Score']])


Top 50 Lemmas per Model:
       Model     Lemma  Occurrences  Min Score  Max Score  Average Score
0      Gemma         "           87    -5.2387    29.7843       4.881663
1      Gemma                     69    -6.5543    30.7499       3.873259
2      Gemma        be           49    -8.1535    12.9251       3.368822
3      Gemma         ,           40    -3.1074    22.9291       4.751890
4      Gemma         I           36    -1.8260     8.3815       2.489083
..       ...       ...          ...        ...        ...            ...
145  Mistral      hell           10    -1.3951    13.1247       3.538010
146  Mistral  naysayer           10    -1.3890     4.9627       0.773670
147  Mistral      Anti            9    -1.3723    16.1613       3.796822
148  Mistral     Trump            9    -1.3723    16.1613       3.796822
149  Mistral        dd            9   -11.1988    12.8812       0.585367

[150 rows x 6 columns]

Top 25 Lemmas with Highest Average Attribution Scores:
            Lemma  